# USAspending Grant Titles: Topic Modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

import re
import string
import time

import stanza
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from gensim.models.phrases import Phrases, Phraser

from sklearn.decomposition import NMF
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#from gensim.models.coherencemodel import CoherenceModel

pd.set_option('display.max_columns', 100)

### Data Ingestion

In [3]:
# read in NSF grants datasets: USAspending, NSF FY16-18

#nsf_g16 = pd.read_csv("../../../data/prd/USAspending_Exploration/NSF/FY16/All_Assistance_PrimeTransactions_2020-10-23_H19M21S10_1.csv")
#nsf_g17 = pd.read_csv("../../../data/prd/USAspending_Exploration/NSF/FY17/All_Assistance_PrimeTransactions_2020-09-25_H17M36S08_1.csv")
nsf_g18 = pd.read_csv("../../../data/prd/USAspending_Exploration/NSF/FY18/All_Assistance_PrimeTransactions_2020-09-25_H18M10S45_1.csv")

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (32,44,56,87,89,91) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#print(nsf_g16.shape)
#print(nsf_g17.shape)
print(nsf_g18.shape)

(25661, 95)


In [5]:
nsf_g18.head()

,assistance_transaction_unique_key,assistance_award_unique_key,award_id_fain,modification_number,award_id_uri,sai_number,federal_action_obligation,total_obligated_amount,non_federal_funding_amount,total_non_federal_funding_amount,face_value_of_loan,original_loan_subsidy_cost,total_face_value_of_loan,total_loan_subsidy_cost,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_funded_by_COVID-19_supplementals_for_overall_aw,obligated_amount_funded_by_COVID-19_supplementals_for_overall_a,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,recipient_duns,recipient_name,recipient_parent_duns,recipient_parent_name,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_code,recipient_city_name,recipient_county_code,recipient_county_name,recipient_state_code,recipient_state_name,recipient_zip_code,recipient_zip_last_4_code,recipient_congressional_district,recipient_foreign_city_name,recipient_foreign_province_name,recipient_foreign_postal_code,primary_place_of_performance_scope,primary_place_of_performance_country_code,primary_place_of_performance_country_name,primary_place_of_performance_code,primary_place_of_performance_city_name,primary_place_of_performance_county_code,primary_place_of_performance_county_name,primary_place_of_performance_state_name,primary_place_of_performance_zip_4,primary_place_of_performance_congressional_district,primary_place_of_performance_foreign_location,cfda_number,cfda_title,assistance_type_code,assistance_type_description,award_description,business_funds_indicator_code,business_funds_indicator_description,business_types_code,business_types_description,correction_delete_indicator_code,correction_delete_indicator_description,action_type_code,action_type_description,record_type_code,record_type_description,highly_compensated_officer_1_name,highly_compensated_officer_1_amount,highly_compensated_officer_2_name,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,last_modified_date
0,4900_1811373_-NONE-_47.049_000,ASST_NON_1811373_4900,1811373,000,NaN,SAI EXEMPT,157966.0,239998.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-04-27,2018,2018-08-01,2021-07-31,49,NATIONAL SCIENCE FOUNDATION (NSF),4900,NATIONAL SCIENCE FOUNDATION,490304,NaN,49.0,NATIONAL SCIENCE FOUNDATION (NSF),4900.0,NATIONAL SCIENCE FOUNDATION,490304.0,NaN,049-2018/2019-0100-000;049-2020/2021-0100-000,049-0100,"41.0: Grants, subsidies, and contributions",NaN,41064767.0,UNIVERSITY OF CINCINNATI,41064767.0,UNIVERSITY OF CINCINNATI,USA,UNITED STATES,2600 CLIFTON AVE,NaN,15000.0,CINCINNATI,61.0,HAMILTON,OH,OHIO,45220.0,2872.0,1.0,NaN,NaN,NaN,Single ZIP Code,USA,UNITED STATES,OH15000,CINCINNATI,61.0,HAMILTON,OHIO,45221-0222,1.0,NaN,47.049,MATHEMATICAL AND PHYSICAL SCIENCES,4,NaN,LIMIT THEOREMS FOR STOCHASTIC PROCESSES AND RA...,NON,NaN,H,NaN,NaN,NaN,A,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/#/award/ASST_NON_1...,2018-05-07 21:09:58.853188+00
1,4900_1231306_-NONE-_47.074_010,ASST_NON_1231306_4900,1231306,010,NaN,SAI EXEMPT,5000000.0,40413810.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,2018-09-14,2018,2013-10-01,2023-09-30,49,NATIONAL SCIENCE FOUNDATION (NSF),4900,NATIONAL SCIENCE FOUNDATION,490808,NaN,49.0,NATIONAL SCIENCE FOUNDATION (NSF),4900.0,NATIONAL SCIENCE FOUNDATION,490808.0,NaN,049-20

### Cleaning Project Titles

In [6]:
# check for null and empty titles

print(sum(nsf_g18['award_description'].isnull()))
print(sum(nsf_g18['award_description'] == ""))

ix = nsf_g18[nsf_g18['award_description'].apply(len)==0].index
print(ix)

0
0
Int64Index([], dtype='int64')


### NLP Processing - Stanza already completed.  Just read in file.

In [14]:
# Create a stanza pipeline: this pipeline will tokenize, determine pos, 
# and then lemmatize the token appropriately.

nlp = stanza.Pipeline(lang='en',processors='tokenize,pos,lemma',tokenize_batch_size=500,lemma_batch_size=500,
                      use_gpu = False)

2020-12-08 11:34:44 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |

2020-12-08 11:34:44 INFO: Use device: cpu
2020-12-08 11:34:44 INFO: Loading: tokenize
2020-12-08 11:34:45 INFO: Loading: pos
2020-12-08 11:34:47 INFO: Loading: lemma
2020-12-08 11:34:47 INFO: Done loading processors!


In [15]:
# Note - I thought about keeping the pos PART if the lemma was "not", ex) isn't --> be, not.  But due to the fact
# that we are using a bag-of-words model that does not take into account word order, this won't matter in the
# topic model.  So, this is not kept.

def token_pos_lemma(doc, pretokened=False, keep_numbers=True):
    
    # This function uses the pipeline to tokenize, find POS, and lemmatize a document
    
    """if pretokened, dont use this function, as it hasnt been adapted for it"""
    
    assert not pretokened #If these are already tokened per another pipeline, this function won't work correctly
    
    new_tokens=[]
     
    processed=nlp(doc)  # this is the line that does the tokenizing, pos, and lemmatizing
    
    for sent in processed.sentences:
        for word in sent.words:
            
            #If its a regular noun, verb, adj, or adverb, keep lemmatized form
            if word.pos in ['NOUN','VERB','ADJ','ADV']:
                new_tokens.append(word.lemma)
            
            #If you decided to retain numbers, their lemma is kept here. Note that number catching isnt perfect by this lemmatizing.
            elif word.pos=='NUM' and keep_numbers:
                new_tokens.append(word.lemma)
            
            #Exact phrases are kept here with no attempt at lemmatization: e.g. mars does not become mars, 
            #and hopefully scientific words e.g. chemicals will be tagged as propn, x, or intj if needed
            elif word.pos in ['PROPN','X','INTJ']: 
                new_tokens.append(word.text)
            
            #Note that no other tokens are kept
    
    return new_tokens

In [16]:
#nsf_g18 = nsf_g18.assign(nlp_title = nsf_g18['award_description'].apply(lambda x: token_pos_lemma(x)))

nlp_titles = nsf_g18['award_description'].apply(lambda x: token_pos_lemma(x))

In [17]:
# save 

#output = open("nsf_g18_titles_NLP.pkl",'wb')
#pickle.dump(nlp_titles, output)
#output.close()

### START HERE to read in Stanza file

In [7]:
# read in file

file_input = open('nsf_g18_titles_NLP.pkl', 'rb')
nlp_titles = pickle.load(file_input)
file_input.close()

In [8]:
len(nlp_titles)

25661

In [9]:
nlp_titles[0:10]

0    [limit, theorem, stochastic, process, random, ...
1                             [biology, x, RAY, laser]
2    [bath, induced, long, range, interaction, diso...
3    [RUI, collaborative, research, nearby, surface...
4    [become, ONLINE, resource, CENTER, ETHICS, edu...
5    [collaborative, research, understand, effect, ...
6    [geometry, transport, symplectic, volume, pres...
7          [crii, cif, CROWDSOURCING, aware, learning]
8    [lter, link, pelagic, community, structure, ec...
9    [career, advance, multi, scale, bayesian, infe...
Name: award_description, dtype: object

In [10]:
nsf_g18.head(10)

,assistance_transaction_unique_key,assistance_award_unique_key,award_id_fain,modification_number,award_id_uri,sai_number,federal_action_obligation,total_obligated_amount,non_federal_funding_amount,total_non_federal_funding_amount,face_value_of_loan,original_loan_subsidy_cost,total_face_value_of_loan,total_loan_subsidy_cost,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_funded_by_COVID-19_supplementals_for_overall_aw,obligated_amount_funded_by_COVID-19_supplementals_for_overall_a,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,recipient_duns,recipient_name,recipient_parent_duns,recipient_parent_name,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_code,recipient_city_name,recipient_county_code,recipient_county_name,recipient_state_code,recipient_state_name,recipient_zip_code,recipient_zip_last_4_code,recipient_congressional_district,recipient_foreign_city_name,recipient_foreign_province_name,recipient_foreign_postal_code,primary_place_of_performance_scope,primary_place_of_performance_country_code,primary_place_of_performance_country_name,primary_place_of_performance_code,primary_place_of_performance_city_name,primary_place_of_performance_county_code,primary_place_of_performance_county_name,primary_place_of_performance_state_name,primary_place_of_performance_zip_4,primary_place_of_performance_congressional_district,primary_place_of_performance_foreign_location,cfda_number,cfda_title,assistance_type_code,assistance_type_description,award_description,business_funds_indicator_code,business_funds_indicator_description,business_types_code,business_types_description,correction_delete_indicator_code,correction_delete_indicator_description,action_type_code,action_type_description,record_type_code,record_type_description,highly_compensated_officer_1_name,highly_compensated_officer_1_amount,highly_compensated_officer_2_name,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,last_modified_date
0,4900_1811373_-NONE-_47.049_000,ASST_NON_1811373_4900,1811373,000,NaN,SAI EXEMPT,157966.0,239998.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-04-27,2018,2018-08-01,2021-07-31,49,NATIONAL SCIENCE FOUNDATION (NSF),4900,NATIONAL SCIENCE FOUNDATION,490304,NaN,49.0,NATIONAL SCIENCE FOUNDATION (NSF),4900.0,NATIONAL SCIENCE FOUNDATION,490304.0,NaN,049-2018/2019-0100-000;049-2020/2021-0100-000,049-0100,"41.0: Grants, subsidies, and contributions",NaN,41064767.0,UNIVERSITY OF CINCINNATI,41064767.0,UNIVERSITY OF CINCINNATI,USA,UNITED STATES,2600 CLIFTON AVE,NaN,15000.0,CINCINNATI,61.0,HAMILTON,OH,OHIO,45220.0,2872.0,1.0,NaN,NaN,NaN,Single ZIP Code,USA,UNITED STATES,OH15000,CINCINNATI,61.0,HAMILTON,OHIO,45221-0222,1.0,NaN,47.049,MATHEMATICAL AND PHYSICAL SCIENCES,4,NaN,LIMIT THEOREMS FOR STOCHASTIC PROCESSES AND RA...,NON,NaN,H,NaN,NaN,NaN,A,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/#/award/ASST_NON_1...,2018-05-07 21:09:58.853188+00
1,4900_1231306_-NONE-_47.074_010,ASST_NON_1231306_4900,1231306,010,NaN,SAI EXEMPT,5000000.0,40413810.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,2018-09-14,2018,2013-10-01,2023-09-30,49,NATIONAL SCIENCE FOUNDATION (NSF),4900,NATIONAL SCIENCE FOUNDATION,490808,NaN,49.0,NATIONAL SCIENCE FOUNDATION (NSF),4900.0,NATIONAL SCIENCE FOUNDATION,490808.0,NaN,049-20

In [11]:
# lowercase tokens

nlp_titles_lower = []
for doc in nlp_titles:
    nlp_titles_lower.append([x.lower() for x in doc])


In [12]:
nlp_titles_lower[0:10]

[['limit',
  'theorem',
  'stochastic',
  'process',
  'random',
  'field',
  'projective',
  'condition'],
 ['biology', 'x', 'ray', 'laser'],
 ['bath',
  'induced',
  'long',
  'range',
  'interaction',
  'disordered',
  'strongly',
  'correlate',
  'optical',
  'lattice'],
 ['rui',
  'collaborative',
  'research',
  'nearby',
  'surface',
  'limit',
  'food',
  'uptake',
  'microscopic',
  'sessile',
  'suspension',
  'feeder',
  '3d',
  'feeding',
  'flow',
  'measurement'],
 ['become',
  'online',
  'resource',
  'center',
  'ethics',
  'education',
  'engineering',
  'science'],
 ['collaborative',
  'research',
  'understand',
  'effect',
  'ploidal',
  'level',
  'response',
  'global',
  'change',
  'plant'],
 ['geometry', 'transport', 'symplectic', 'volume', 'preserve', 'dynamics'],
 ['crii', 'cif', 'crowdsourcing', 'aware', 'learning'],
 ['lter',
  'link',
  'pelagic',
  'community',
  'structure',
  'ecosystem',
  'dynamics',
  'production',
  'regime',
  'change',
  'northea

In [13]:
# remove stop words - Needs some work...removes US because it became "us"

stop_wds = STOP_WORDS  # from Spacy

nlp_titles_l_nosw = []
for doc in nlp_titles_lower:    
    nlp_titles_l_nosw.append([word for word in doc if word not in stop_wds]) 

#df = drop_empties(df, "stopwds_removed")

In [14]:
nlp_titles_l_nosw[0:10]

[['limit',
  'theorem',
  'stochastic',
  'process',
  'random',
  'field',
  'projective',
  'condition'],
 ['biology', 'x', 'ray', 'laser'],
 ['bath',
  'induced',
  'long',
  'range',
  'interaction',
  'disordered',
  'strongly',
  'correlate',
  'optical',
  'lattice'],
 ['rui',
  'collaborative',
  'research',
  'nearby',
  'surface',
  'limit',
  'food',
  'uptake',
  'microscopic',
  'sessile',
  'suspension',
  'feeder',
  '3d',
  'feeding',
  'flow',
  'measurement'],
 ['online',
  'resource',
  'center',
  'ethics',
  'education',
  'engineering',
  'science'],
 ['collaborative',
  'research',
  'understand',
  'effect',
  'ploidal',
  'level',
  'response',
  'global',
  'change',
  'plant'],
 ['geometry', 'transport', 'symplectic', 'volume', 'preserve', 'dynamics'],
 ['crii', 'cif', 'crowdsourcing', 'aware', 'learning'],
 ['lter',
  'link',
  'pelagic',
  'community',
  'structure',
  'ecosystem',
  'dynamics',
  'production',
  'regime',
  'change',
  'northeast',
  'shel

### NMF Topic Model

In [15]:
# input needed for NMF (from Scikit-Learn) is one string per document (not a list of strings)

text = []

for doc in nlp_titles_l_nosw:
    text.append(" ".join(doc))


In [16]:
text[0:10]

['limit theorem stochastic process random field projective condition',
 'biology x ray laser',
 'bath induced long range interaction disordered strongly correlate optical lattice',
 'rui collaborative research nearby surface limit food uptake microscopic sessile suspension feeder 3d feeding flow measurement',
 'online resource center ethics education engineering science',
 'collaborative research understand effect ploidal level response global change plant',
 'geometry transport symplectic volume preserve dynamics',
 'crii cif crowdsourcing aware learning',
 'lter link pelagic community structure ecosystem dynamics production regime change northeast shelf',
 'career advance multi scale bayesian inference learn massive data']

In [17]:
# Helper function to print topics
# function slightly modified from https://nlpforhackers.io/topic-modeling/

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):  # loop through each row of H.  idx = row index.  topic = actual row
        print("\nTopic %d:" % (idx))
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
        print_list = [(vectorizer.get_feature_names()[i], topic[i])  
                        for i in topic.argsort()[:-top_n - 1:-1]]
        for item in print_list:
            print(item)

In [18]:
# create document-term matrix - TFIDF 

tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase=False, max_features=None)
tf_idf = tfidf_vectorizer.fit_transform(text)

In [21]:
t1 = time.time()
nmf_model = NMF(n_components=20, random_state = 0)
nmf_model.fit_transform(tf_idf)
t2 = time.time()
print(f"  Model time: {t2-t1}")

  Model time: 4.867626667022705


In [22]:
print_topics(nmf_model, tfidf_vectorizer, 10)


Topic 0:
('grfp', 2.521456884188997)
('graduate', 2.451684523695058)
('fellowship', 2.4061447633820974)
('program', 2.1952134083044386)
('research', 1.028959314756437)
('scholar', 0.029614673113355642)
('earthquake', 0.025764591416055124)
('scholarship', 0.02234071212218398)
('gradute', 0.021175935986869187)
('2017', 0.017218389366369186)

Topic 1:
('research', 3.8790949897230833)
('collaborative', 3.8446439538565387)
('medium', 0.31763598767005136)
('change', 0.23038230949810415)
('community', 0.20338247055824252)
('climate', 0.19508555191244423)
('wave', 0.18371685576598643)
('ocean', 0.17969971770370927)
('rapid', 0.17008775987451524)
('mechanism', 0.16287006699957643)

Topic 2:
('phase', 2.4496016351403527)
('ii', 1.9154397405474506)
('sbir', 1.5344178916537081)
('ucrc', 1.0749663413748753)
('center', 0.703586099933757)
('university', 0.30322802997114795)
('advanced', 0.2279688409656358)
('iucrc', 0.21325819811104788)
('technology', 0.20229691475568992)
('platform', 0.156087706134